In [13]:
# Made with Julia 1.7.3
using DataFrames, CSV
# Prevent Makie from open an external window to display graphs
# Makie.inline!(true)

## Read in Data
### File Parser Function Definitions

In [14]:
"""
    readtransformfile(filename)

Create a dataframe with transform tracking data from `filename`
"""
function readtransformfile(filename)
    # Get the length of the file
    filelength = filesize(filename)
    # Construct empty matrix matching the length of the file
    # One column is one log, ie col 1 is:
    # controller pos [x,y,z], rot [x,y,z,w], headset pos [x,y,z], rot [x,y,z,w] (14 values)
    data_matrix = Matrix{Float32}(undef, 14, filelength ÷ 4 ÷ 14)

    # Open data file and read the file into the array
    open(filename) do dataFile
        read!(dataFile, data_matrix)
    end

    # BAD WAY TO DO IT (hcat generates tooons of intermediate arrays):
    # while !eof(dataFile)
    #     data_matrix = hcat(data_matrix, [x for x in reinterpret(Float32, read(dataFile, 4*14))])
    # end

    # Rotate the matrix so the columns are one variable, ie col1 = controller position x component
    data_matrix = permutedims(data_matrix, (2,1))

    # Construct the data frame
    col_names = ["c_pos_x", "c_pos_y", "c_pos_z",
                "c_rot_x", "c_rot_y", "c_rot_z", "c_rot_w",
                "h_pos_x", "h_pos_y", "h_pos_z",
                "h_rot_x", "h_rot_y", "h_rot_z", "h_rot_w"]
    return DataFrame(data_matrix, col_names)
end

readtransformfile

In [21]:
"""
    readlogfile(filename)

Create a dataframe with log data from `filename`
"""
function readlogfile(filename)
    # Vectors to hold the parsed components of each log
    entities = String[]
    tags = String[]
    abstimes = Vector{Tuple{Int,Int,Int}}()
    reltimes = Float64[]
    messages = String[]

    # Open data file and iterate over each line
    open(filename) do logFile
        for line in eachline(logFile)
            # If the line has a |, then it's probably a log
            if contains(line, "|")
                # Parse the line for the different parts of the log using regex
                entity_match = match(r"^[A-Z ]+", line)
                push!(entities, strip(entity_match.match))
                tag_match = match(r"[A-Z]+ ?[A-Z]*", line, length(entity_match.match)+1)
                push!(tags, rstrip(tag_match.match))
                abstime_match = match(r"(\d{1,2}):(\d{1,2}):(\d{1,2})", line, tag_match.offset + length(tag_match.match))
                push!(abstimes, Tuple(parse.(UInt8, abstime_match.captures)))
                reltime_match = match(r"[\d\.]+", line, abstime_match.offset + length(abstime_match.match))
                push!(reltimes, parse(Float64, reltime_match.match))
                message_match = match(r"[^\|]+$", line, reltime_match.offset + length(reltime_match.match))
                push!(messages, message_match.match)
            end
        end
    end

    # Construct and return the dataframe using the vectors
    return DataFrame(entity=entities,
                     tag=tags,
                     abstime=abstimes,
                     reltime=reltimes,
                     message=messages)
end

readlogfile

### Read in files, put dataframes in dict
this currently does not support handling Guest logs
User 121 has only an empty csv file on the server, not sure why
deleting locally

In [24]:
### Create Dictionaries to pair user IDs to dataframes of log data
# Dictionary linking user names to dataframse with log data
user_logs = Dict{Int,DataFrame}()
# Dictionary linking user names to dataframes with their transform data
user_transforms = Dict{Int,DataFrame}()

### Parse the log files
# Loop through all the files in the Data directory
for filename in readdir(raw"..\Data\RawLogs", join=true)
    # Find substring User_##, use as key to df in dictionary
    key = 0
    match_obj = match(r"User_(\d{1,3})", filename)
    if !isnothing(match_obj)
        key = parse(Int, match_obj[1])
    else # Skip Guest logs
        continue
    end

    # For the files that contain binary serialized transform data
    if endswith(filename, "TransformTracking.dat")
        df = readtransformfile(filename)
        user_transforms[key] = df
    end
    
    # For the general log files
    if endswith(filename, ".log")
        df = readlogfile(filename)
        user_logs[key] = df
    end
end

In [23]:
# Sample of dataframe rows
show(first(user_transforms[88], 10), allcols=true)
show(first(user_logs[88], 10), allcols=true)

10×14 DataFrame
 Row │ c_pos_x     c_pos_y     c_pos_z      c_rot_x       c_rot_y      c_rot_z      c_rot_w      h_pos_x      h_pos_y     h_pos_z     h_rot_x      h_rot_y      h_rot_z     h_rot_w    
     │ Float32     Float32     Float32      Float32       Float32      Float32      Float32      Float32      Float32     Float32     Float32      Float32      Float32     Float32    
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │  3.82827     3.62827     0.0          0.0           0.0         -0.0         -0.0         -0.0          1.0         0.0         0.0          0.0         -0.0        -0.0
   2 │ -0.0         1.0         0.00109844  -0.00997893   -0.772436     0.192555    -0.0588141   -0.205649     0.626851    0.749204    0.0475877   -0.024524    -0.0225267  -0.116913
   3 │  0.104904    0.0400868  -0.986772     0.000458047  -0.00997893  -0

## Export the Data

In [25]:
for key in keys(user_logs)
    CSV.write("../Data/Dataframes/Logs/$(key)_logs.csv", user_logs[key])
end

for key in keys(user_transforms)
    CSV.write("../Data/Dataframes/Transforms/$(key)_transforms.csv", user_transforms[key])
end